In [46]:
# DATA LOADER
import os
import torchaudio
import torch
import random

y_mask=True
n_noise = -1
REFERENCE_CHANNEL = 0
SAME_LENGTH = True
THR_S = 0.5 # Threshold for speech IRM
THR_N = 0.5
N_PATH = "/project/data_asr/CHiME5/data/librenoise/free-sound/"#"/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/noise/free-sound/"
S_PATH = "/project/data_asr/CHiME5/data/librenoise/dev/dev-clean/"#"/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/LibriSpeech/dev-clean/"
MODEL_SAVE_PATH = "/project/data_asr/CHiME5/data/librenoise/models/"
def load_noise(N_PATH=N_PATH):
    noise = []
    for file in  os.listdir(N_PATH):
        if file[-4:] == ".wav":
            sound, _ = torchaudio.load(N_PATH+file)
            noise.append(sound)
    return noise


def load_speech(S_PATH=S_PATH):
    speech = []
    for folder in  os.listdir(S_PATH):
        if os.path.isdir(S_PATH+folder):
            for subfolder in os.listdir(S_PATH+folder):
                if os.path.isdir(S_PATH+folder+"/"+subfolder):
                    for file in os.listdir(S_PATH+folder+"/"+subfolder):
                        if file[-5:] == ".flac":
                            sound, _ = torchaudio.load(S_PATH+folder+"/"+subfolder+"/"+file)
                            if SAME_LENGTH:
                                try:
                                    sound = torch.narrow(sound,1,0,50000)# Narrow to 50000
                                except Exception:
                                    # add zeros to make sound 50000 long
                                    len_sound = sound.shape[1]
                                    add_zeros = 50000 - len_sound
                                    add_zeros = torch.zeros(add_zeros).reshape(1,-1)
                                    sound = torch.concat([sound,add_zeros],dim=1)
                            speech.append(sound)
    return speech

def add_noise_to_speech(speech, noise, ratio1: float, ratio2: float):
    X = []
    X2 = []
    newNoise = []
    for sample in speech:
        len_speech = sample.shape[1]
        sample_noise = random.choice(noise)
        while sample_noise.shape[1]<len_speech:
            sample_noise = torch.concat([sample_noise,sample_noise],dim=1)# Repeat to ensure noise is longer than speech
        sample_noise = torch.narrow(sample_noise,1,0,len_speech)# Shorten noise to same length as speech
        x = torch.add(sample,sample_noise*ratio1)# Same Ratio 1:1
        x2 = torch.add(sample,sample_noise*ratio2)# Same Ratio 1:1
        sample_noise = torch.narrow(sample_noise,1,0,len_speech)
        X.append(x)
        X2.append(x2)
        newNoise.append(sample_noise)
    return X, X2, newNoise    

speech = load_speech()
noise = load_noise()
# if n_noise>0:
#     noise = noise[:n_noise]
#print(torch.stack(noise)[0].shape)
#noise = noise[5:6]
#print(torch.stack(noise)[0].shape)
mix1, mix2, noise = add_noise_to_speech(speech, noise, 0.2, 0.5)

# STFT
N_FFT = 1024
N_HOP = 256

stft = torchaudio.transforms.Spectrogram(
    n_fft=N_FFT,
    hop_length=N_HOP,
    power=None,
)
istft = torchaudio.transforms.InverseSpectrogram(n_fft=N_FFT, hop_length=N_HOP)

stfts_mix = []
for n in range(0,len(mix1)):
    x_new = torch.concat([stft(mix1[n]),stft(mix2[n])],dim=0)
    stfts_mix.append(x_new)

    
stfts_clean = []
for y in speech:
    y_new = stft(y)
    y_new = y_new.reshape(513,-1)
    stfts_clean.append(y_new)

stfts_noise = []
i = 0
for n in noise:
    try:
        n_new = stft(n)
        stfts_noise.append(n_new.reshape(513,-1))
    except Exception: #sometimes noises are very short, e.g. noise[697]
        continue
def get_irms(stft_clean, stft_noise):
    mag_clean = stft_clean.abs() ** 2
    mag_noise = stft_noise.abs() ** 2
    irm_speech = mag_clean / (mag_clean + mag_noise)
    irm_noise = mag_noise / (mag_clean + mag_noise)
    return irm_speech[REFERENCE_CHANNEL], irm_noise[REFERENCE_CHANNEL]

Y = []
for n in range(0,len(noise)):
    irm_speech, irm_noise = get_irms(stfts_clean[n].unsqueeze(0), stfts_noise[n].unsqueeze(0))
    irm_speech = (irm_speech>THR_S).float()
    irm_noise = (irm_noise>THR_N).float()
    Y.append(torch.cat((irm_speech.unsqueeze(0),irm_noise.unsqueeze(0)),0))

X_h = []
stfts_mix_s = torch.stack(stfts_mix)

for i in range(0, len(stfts_mix)):
    X_i = []
    for j in range (0,2):
        X_i.append(torch.cat((stfts_mix_s[i][j].real.unsqueeze(0),stfts_mix_s[i][j].imag.unsqueeze(0)),0))
    X_h.append(torch.cat((X_i[0],X_i[1]),0))
speech_s = torch.stack(speech)
noise_s = torch.stack(noise)
mix1_s = torch.stack(mix1)
Y = torch.stack(Y)
X = torch.stack(X_h)
if y_mask == False:
    Y = speech_s # NEW
#return X,Y,speech_s,noise_s,mix1_s,stfts_mix_s
X_all = X
X_all = X_all.view(2703,513,196,4)
speech_all= speech
noise_all = noise
mix_all = mix1_s
stft_all = stfts_mix_s

In [53]:
torchaudio.save("./outputs/noise_all.wav",torch.stack(noise_all)[50],16000)
torchaudio.save("./outputs/noise_all.wav",speech_s[50],16000)
torchaudio.save("./outputs/mix_all.wav",mix1_s[50],16000)

In [44]:
print(torch.stack(noise_all)[5])
print(torch.stack(noise_one)[10])
torchaudio.save("noise_all.wav",torch.stack(noise_all)[5],16000)
torchaudio.save("noise_one.wav",torch.stack(noise_one)[10],16000)


tensor([[0.0000e+00, 0.0000e+00, 3.0518e-05,  ..., 3.4317e-01, 4.1779e-01,
         6.4844e-01]])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1898, 0.2145, 0.2414]])


## GOAL

In [20]:
# ONE
import os
import torchaudio
import torch

y_mask=True
n_noise = -1
REFERENCE_CHANNEL = 0
SAME_LENGTH = True
THR_S = 0.5 # Threshold for speech IRM
THR_N = 0.5
N_PATH = "/project/data_asr/CHiME5/data/librenoise/free-sound/"#"/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/noise/free-sound/"
S_PATH = "/project/data_asr/CHiME5/data/librenoise/dev/dev-clean/"
def load_noise(N_PATH=N_PATH):
    noise = []
    for file in  os.listdir(N_PATH):
        if file[-4:] == ".wav":
            sound, _ = torchaudio.load(N_PATH+file)
            noise.append(sound)
    return noise


def load_speech(S_PATH=S_PATH):
    speech = []
    for folder in  os.listdir(S_PATH):
        if os.path.isdir(S_PATH+folder):
            for subfolder in os.listdir(S_PATH+folder):
                if os.path.isdir(S_PATH+folder+"/"+subfolder):
                    for file in os.listdir(S_PATH+folder+"/"+subfolder):
                        if file[-5:] == ".flac":
                            sound, _ = torchaudio.load(S_PATH+folder+"/"+subfolder+"/"+file)
                            if SAME_LENGTH:
                                try:
                                    sound = torch.narrow(sound,1,0,50000)# Narrow to 50000
                                except Exception:
                                    # add zeros to make sound 50000 long
                                    len_sound = sound.shape[1]
                                    add_zeros = 50000 - len_sound
                                    add_zeros = torch.zeros(add_zeros).reshape(1,-1)
                                    sound = torch.concat([sound,add_zeros],dim=1)
                            speech.append(sound)
    return speech

def add_noise_to_speech(speech, noise):
    X = []
    for sample in speech:
        len_speech = sample.shape[1]
        sample_noise = torch.concat([noise[6],noise[6],noise[6],noise[6]],dim=1)# Repeat to ensure noise is longer than speech
        sample_noise = torch.narrow(sample_noise,1,0,len_speech)# Shorten noise to same length as speech
        x = torch.add(sample,sample_noise*0.2)# Same Ratio 1:1
        X.append(x)
    return X    

def add_noise_to_speech2(speech, noise):
    X = []
    for sample in speech:
        len_speech = sample.shape[1]
        sample_noise = torch.concat([noise[6],noise[6],noise[6],noise[6]],dim=1)# Repeat to ensure noise is longer than speech
        sample_noise = torch.narrow(sample_noise,1,0,len_speech)# Shorten noise to same length as speech
        x = torch.add(sample,sample_noise*0.5)# Same Ratio 1:1
        X.append(x)
    return X 

def get_one_noise(speech, noise):# Make Noise 6 same length as respective speech
    X = []
    for sample in speech:
        len_speech = sample.shape[1]
        sample_noise = torch.concat([noise[6],noise[6],noise[6],noise[6]],dim=1)# Repeat noise to ensure noise is longer than speech
        sample_noise = torch.narrow(sample_noise,1,0,len_speech)# Shorten noise to same length as speech
        x = sample_noise
        X.append(x)
    return X

# STFT
N_FFT = 1024
N_HOP = 256

stft = torchaudio.transforms.Spectrogram(
    n_fft=N_FFT,
    hop_length=N_HOP,
    power=None,
)

speech = load_speech()
noise = load_noise()
X = add_noise_to_speech(speech, noise)
X_e = X
X2 = add_noise_to_speech2(speech,noise)
noise = get_one_noise(speech,noise)
stfts_mix = []
for n in range(0,len(X)):
    x_new = torch.concat([stft(X[n]),stft(X2[n])],dim=0)
    stfts_mix.append(x_new)

    
stfts_clean = []
for y in speech:
    y_new = stft(y)
    y_new = y_new.reshape(513,-1)
    stfts_clean.append(y_new)

stfts_noise = []
for n in noise:
    try:
        n_new = stft(n)
        stfts_noise.append(n_new.reshape(513,-1))
    except Exception: #sometimes noises are very short, e.g. noise[697]
        continue

def get_irms(stft_clean, stft_noise):
    mag_clean = stft_clean.abs() ** 2
    mag_noise = stft_noise.abs() ** 2
    irm_speech = mag_clean / (mag_clean + mag_noise)
    irm_noise = mag_noise / (mag_clean + mag_noise)
    return irm_speech[REFERENCE_CHANNEL], irm_noise[REFERENCE_CHANNEL]

trainY = []
for n in range(0,len(noise)):
    irm_speech, irm_noise = get_irms(stfts_clean[n].unsqueeze(0), stfts_noise[n].unsqueeze(0))
    irm_speech = (irm_speech>THR_S).float()
    irm_noise = (irm_noise>THR_N).float()
    trainY.append(torch.cat((irm_speech.unsqueeze(0),irm_noise.unsqueeze(0)),0))

X = torch.stack(stfts_mix)
Y = torch.stack(trainY)

# Add second channel to one channel => (513,196,2) -> (513,196,4)
def prep_xij(trainX,i,j):
    real_part = trainX[i][j].real
    imag_part = trainX[i][j].imag
    return torch.cat((real_part.unsqueeze(2),imag_part.unsqueeze(2)),2)

def prep_x(trainX,i):
    x = []
    for j in range(0,2):# + Iterate over channels
        x.append(prep_xij(trainX,i,j))
    x = torch.stack(x,dim=2)
    x = x.reshape(513,196,4)
    return x

X_4 = []
for i in range(0,len(X)):
    x = X[i]
    x = prep_x(X,i)
    X_4.append(x)
X_4 = torch.stack(X_4)

#return X_4,Y,speech,X_e, X
print(X_4.shape)
X_one = X_4
speech_one = speech
noise_one = noise
mix_one = X_e
stft_one = stfts_mix

torch.Size([2703, 513, 196, 4])


In [17]:
print(X_all.shape)
print(X_one.shape)
print(X_all[0][0][0:2])
print(X_one[0][0][0:2])

torch.Size([2703, 513, 196, 4])
torch.Size([2703, 513, 196, 4])
tensor([[-0.8135, -0.7360, -0.3900,  0.0422],
        [ 0.1335,  0.0230, -0.1164, -0.2497]])
tensor([[-0.1300,  0.0000, -0.1298,  0.0000],
        [-0.1003,  0.0000, -0.1003,  0.0000]])


In [26]:
print(torch.stack(noise_all).shape)
print(torch.stack(noise_one).shape)
print(noise_all[0])
print(noise_one[0])

torch.Size([2703, 1, 50000])
torch.Size([2703, 1, 50000])
tensor([[-3.0518e-05, -3.0518e-05,  0.0000e+00,  ...,  5.6854e-02,
          2.7469e-01,  3.2697e-01]])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1898, 0.2145, 0.2414]])


In [ ]:
torchaudio.save("mix1.wav",mix1[0].reshape(1,-1),16000)
torchaudio.save("mix2.wav",mix2[0].reshape(1,-1),16000)
torchaudio.save("mX.wav",X[0].reshape(1,-1),16000)
torchaudio.save("mX2.wav",X2[0].reshape(1,-1),16000)

Ergebnisse:
- Speech ist identisch
- NOISE IST ANDERS!!!
- X ist unterschiedlich => Y auch